In [ ]:
# Acknowledge
# This is a Group project with ZhiWei Zhan, Boyangzhang, Jiayou Qu. The pulished code may have very similar content with their published code.
# Some codes are inspired by open source project and ChatGPT

# This step 1 code has done:
# 1, load the 10K report data and ESG rating data
# 2, match the 10k report company and ESG rating company
# 3, Split the report in to sentence

In [1]:
# import necessary libiary
import pandas as pd 
import requests 
import json
from sec_edgar_api import EdgarClient
import re
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import html2text

d:\anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[nltk_data] Error loading punkt: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>


In [2]:
#Load ESG rating data
df=pd.read_excel('D:/UCL/workspace/final/data/Software.xlsx')
df = df[1:]
df['Unnamed: 1'] = df['Unnamed: 1'].str.replace(r' \b\w{2} Equity\b', '', regex=True)
df = df.dropna(subset=['Unnamed: 4','Unnamed: 5','Unnamed: 6'], how='any')
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
2,8x8 Inc,EGHT,Software and Tech Services,United States,7.89575,7.153529,7.192728,6.888117,6.835288,7.191468,7.220757,6.364528
6,ANSYS Inc,ANSS,Software and Tech Services,United States,6.936045,6.570246,6.713562,6.919331,6.83808,6.274809,6.394897,6.593236
7,Accenture PLC,ACN,Software and Tech Services,United States,7.970041,8.126698,7.616173,7.835817,7.84304,7.884892,8.175945,8.108513
10,Adobe Inc,ADBE,Software and Tech Services,United States,7.873047,7.886715,8.034689,7.706401,7.2574,7.348884,6.827739,6.466184
18,Alteryx Inc,AYX,Software and Tech Services,United States,4.949689,5.876694,5.92341,6.352611,6.372025,6.079855,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
363,Yext Inc,YEXT,Software and Tech Services,United States,5.973114,6.608333,6.414529,6.224678,6.324669,4.214297,NaN,NaN
367,Zoom Video Communications Inc,ZM,Software and Tech Services,United States,5.791002,5.85837,5.305337,NaN,NaN,NaN,NaN,NaN
368,ZoomInfo Technologies Inc,ZI,Software and Tech Services,United States,5.618888,5.3146,5.477425,NaN,NaN,NaN,NaN,NaN
370,Zscaler Inc,ZS,Software and Tech Services,United States,5.023742,5.014425,5.234458,5.335603,5.296031,NaN,NaN,NaN


In [3]:
#load the 10k report data in following cells
CompanyTickers = requests.get('https://www.sec.gov/files/company_tickers.json',
                              headers={'User-Agent':"test@gmail.com"})

CompanyTicker = open('CompanyTickers.json','w+')
CompanyTicker.write(json.dumps(CompanyTickers.json()))
CompanyTicker.close()

CompanyData = pd.DataFrame.from_dict(CompanyTickers.json(), orient='index')
display(CompanyData)

,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1045810,NVDA,NVIDIA CORP
3,1652044,GOOGL,Alphabet Inc.
4,1018724,AMZN,AMAZON COM INC
...,...,...,...
10159,1947861,NMHIW,Nature's Miracle Holding Inc.
10160,1819794,HTOOW,Fusion Fuel Green PLC
10161,922247,CYTHW,"Cyclo Therapeutics, Inc."
10162,1212545,WAL-PA,WESTERN ALLIANCE BANCORPORATION


In [4]:
def FindCompanyCik(Ticker):
    for i in CompanyTickers.json().keys():
        if CompanyTickers.json()[i]['ticker'] ==Ticker:
            return CompanyTickers.json()[i]['cik_str']

In [5]:
def checkExistingof10k (Companycik):
    edgar = EdgarClient(user_agent="test@gmail.com")
    try:
        test = edgar.get_company_concept(cik=Companycik, taxonomy="us-gaap", tag="AccountsPayableCurrent")
        reportRecord = pd.DataFrame.from_dict(test['units']['USD'])
        pass
    except:
        return False
    type='10-K'
    if type in reportRecord['form'].values:
        return True
    else:
        return False 

In [6]:
def SearchIndexContents(text):
    IndexContent = re.compile(r'((ITEM\ *\d+\.\ *\s*(.*?)\n))|(Item\ *\d+\.\ *\s*(.*?)\n)', re.IGNORECASE).finditer(text)


    ItemIndex_df = pd.DataFrame([(match.group().strip(), match.start(), match.end()) for match in IndexContent])
    ItemIndex_df.columns = ['Item', 'start', 'end']
    if len(ItemIndex_df) > 16:
        ItemIndex_df = ItemIndex_df[(len(ItemIndex_df)-16):len(ItemIndex_df)]
    ItemIndex_df = ItemIndex_df.reset_index(drop=True)
    display(ItemIndex_df)
    return ItemIndex_df

In [7]:
def Extractedtext(dataframe:pd,Item:int,report:str):
    start_items = dataframe[dataframe['Item'].str.contains(f"ITEM\s+{Item}", case=False, regex=True)]
    end_items = dataframe[dataframe['Item'].str.contains(f"ITEM\s+{Item+1}", case=False, regex=True)]
    if (start_items.empty==True) or (end_items.empty==True):
        #print('All in empty')
        return ""
    else:
        start_index = start_items.iloc[0]['start']
        end_index = end_items.iloc[0]['start']
        extracted_text = report[start_index:end_index].strip()
        extracted_text = re.sub(r'\*|\_|\||\#|\-', '', extracted_text)
        extracted_text = re.sub(r'\n', ' ', extracted_text)
        return(extracted_text)

In [8]:
def ExtractESG(text):
    ReportIndex = SearchIndexContents(text)

    item1 = Extractedtext(ReportIndex,1,text)
    item3 = Extractedtext(ReportIndex,3,text)
    item7 = Extractedtext(ReportIndex,7,text)
    item10 = Extractedtext(ReportIndex,10,text)
    item11 = Extractedtext(ReportIndex,11,text)
    item12 = Extractedtext(ReportIndex,12,text)
    All = item1 + ' '+ item3 +' '+ item7 +' '+ item10 +' '+ item11 +' '+ item12
    Split = sent_tokenize(All)
    
    return Split

In [9]:
def Download10kdata(companyCik,CompanyName,CompanyRec):
    edgar = EdgarClient(user_agent="test@gmail.com")
    try:
        test = edgar.get_company_concept(cik=companyCik, taxonomy="us-gaap", tag="AccountsPayableCurrent")
    except:
        print("pass")
    else:
        reportRecord = pd.DataFrame.from_dict(test['units']['USD'])
        Company10K = reportRecord[reportRecord['form']=='10-K']
        Company10K=Company10K.drop_duplicates(['fy'])
        Company10K=Company10K.set_index('fy')
        for i,row in Company10K.iterrows():  
            accn = row['accn']
            print(f'{CompanyName}-{accn}')
            accnid = accn.replace('-','')

            try:
                Report_10K = requests.get(f'https://www.sec.gov/Archives/edgar/data/{companyCik}/{accnid}/{accn}.txt',
                                headers={'User-Agent':"test@gmail.com"})
                raw_10k = Report_10K.text
                clear10K = html2text.html2text(raw_10k)
                #print(clear10K)
                split = ExtractESG(clear10K)
                #print(split)
                # Create a folder for save data
                df = pd.DataFrame(split)
                df.to_csv(f"D:/UCL/workspace/final/data/{CompanyName}-{i}.csv",index=False)
                CompanyRec.loc[len(CompanyRec.index)]=[CompanyName,i,companyCik, accn,'True',len(df.index)]
                pass
            except:
                CompanyRec.loc[len(CompanyRec.index)]=[CompanyName,i,companyCik, accn,'False',0]
                pass

In [10]:
# counting company quantity in SEC list
count = 0
industry = []
for name in df['Unnamed: 1']:
    if name in CompanyData['ticker'].values:
        count = count + 1
        industry.append(name)
        
print(industry) 
print(count)
Mix=industry

['EGHT', 'ANSS', 'ACN', 'ADBE', 'APPF', 'TEAM', 'ADSK', 'ADP', 'BILL', 'BAND', 'BZUN', 'BSY', 'BL', 'BAH', 'BOX', 'BR', 'CDW', 'CDNS', 'CDLX', 'CHKP', 'NET', 'CSGP', 'CTSH', 'CVLT', 'CNDT', 'CRWD', 'CYBR', 'DXC', 'DAY', 'DOCU', 'DT', 'EPAM', 'ESTC', 'ENV', 'EFX', 'EEFT', 'EVH', 'EXLS', 'FDS', 'FICO', 'FI', 'FTNT', 'GOTU', 'IT', 'GEN', 'GPN', 'GLOB', 'GDOT', 'HCAT', 'HUBS', 'INTU', 'FROG', 'WLY', 'KBR', 'KC', 'LPSN', 'RAMP', 'MSCI', 'MKTX', 'MA', 'MSFT', 'MDB', 'MCO', 'MORN', 'NTNX', 'OKTA', 'ORCL', 'PRO', 'PAGS', 'PANW', 'PYPL', 'PAYX', 'PAYC', 'PCTY', 'QTWO', 'QLYS', 'ROP', 'SPGI', 'SPSC', 'SABR', 'CRM', 'NOW', 'SMAR', 'SWI', 'STNE', 'SNCR', 'SNPS', 'TENB', 'TRU', 'TWLO', 'U', 'UPLD', 'VRNS', 'VEEV', 'VRNT', 'VRSK', 'VRRM', 'V', 'WNS', 'WU', 'WDAY', 'YEXT', 'ZM', 'ZI', 'ZS', 'EGAN']
106


In [11]:
#obtain the final result
downloadRec = pd.DataFrame(columns=['CompanyTicker','Year','CIK', 'ACCN','Status','CSV.Length'])
for item in Mix:
        CompanyName = item
        Companycik = FindCompanyCik(CompanyName)
        Download10kdata(Companycik,CompanyName,downloadRec)
        
print(downloadRec)

EGHT-0001136261-12-000328


,Item,start,end
0,ITEM 3. LEGAL PROCEEDINGS **From time to,124900,124941
1,ITEM 4. MINE SAFETY DISCLOSURES **Not applicab...,127656,127716
2,ITEM 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,127717,127795
3,ITEM 6. SELECTED FINANCIAL DATA,129664,129696
4,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,131567,131639
5,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,183504,183556
6,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,301412,301484
7,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERSAND C...,304579,304645
8,Item 10.\n\n69,305102,305115
9,"Item 5.05 of Form 8-K regarding an amendment to,",305458,305507


EGHT-0001136261-13-000259
EGHT-0001136261-14-000239


,Item,start,end
0,ITEM 4. MINE SAFETY DISCLOSURES **Not applicable.,149017,149067
1,ITEM 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,149078,149156
2,ITEM 6. SELECTED FINANCIAL DATA,151021,151053
3,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,152361,152433
4,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,210853,210905
5,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,343888,343960
6,"ITEM 10. DIRECTORS, EXECUTIVE OFFICERSAND CORP...",347639,347702
7,Item 10.\n\nWe have adopted a Code of Conduct ...,348159,348244
8,"Item 5.05 of Form 8-K regarding an amendment to,",348504,348553
9,ITEM 11. EXECUTIVE COMPENSATION,348729,348761


EGHT-0001136261-15-000166
